## Agile Hardware Design
***
# Chisel Grab Bag

## Prof. Scott Beamer
### sbeamer@ucsc.edu

## [CSE 293](https://classes.soe.ucsc.edu/cse293/Winter22/)

## Plan for Today

* Recap how Chisel "works"
* Tips for common mistakes to avoid

## Loading The Chisel Library Into a Notebook

In [ ]:
val path = System.getProperty("user.dir") + "/../resource/chisel_deps.sc"
interp.load.module(ammonite.ops.Path(java.nio.file.FileSystems.getDefault().getPath(path)))

In [ ]:
import chisel3._
import chisel3.util._
import chisel3.tester._
import chisel3.tester.RawTester.test

## Recap of What Chisel/Scala Does

* Your "Chisel" design is a valid Scala program (otherwise will get a Scala compile error)
  * Chisel is a Scala library, and we treat it like an _embedded domain-specific language_
* While executing, any Chisel object referenced/constructed is instantiated
  * Includes even a literal (e.g. `4.U`) or a slight tweak (e.g. `~io.in`)
  * Every object has _inputs_ and/or _outputs_
  * Under the hood, those objects are tracked (e.g. inside `extends Module`)
* Chisel connections (`:=` and `<>`) induce side-effects on the Chisel objects
  * Changes inputs by connecting them to outputs
* _Summary:_ think of your Chisel design as a Scala program that _instantiates_ Chisel things and _connects_ them
  * Much of the parameterizability/flexibility is coming from the Scala program
* Your goal is to properly connect things to the inputs/outputs of modules
  * Tools will prune components that are uncreachable from an input or output

## Hardware Designs Are _Static Structurally_ with _Dynamic Signals_

* Hardware's connectivity/structure is _static_ after _elaboration_
  * Even though we are only simulating and not manufacturing the physical designs, the design is unchanged
  * A mux input can change its output, but it has static connections externally
* A wire can carry different values in different cycles (in simulation or real world), but the wire's endpoint connections are unchanged
  * A wire has no internal state, and directly propagates its input to its output
  * A wire changing value over "time" is caused by its input changing over time
* A _register_ (or memory) has internal state, but only changes value at the rising clock edge
  * At rising edge, input value becomes output (and internal state) value
  * Typically in Chisel we don't explicitly show clock, so sometimes easy to loose track of when things change

## Mutability, but When & Where?

* _Mutability_ (e.g. `var` in Scala) impacts how your Scala program will behave
  * Chisel tools mostly can't tell if you declare things with `val` or `var`
  * Do not confuse mutability in Scala with hardware signal values varying in time
* Example of implementing a counter
    * **Incorrect**
    ```scala
    var counter = 0.U
    counter = counter + 1.U
    ```
        * `counter` (in elaborated hardware) will always be `0 + 1`
    * **Better**
    ```scala
    val counter = Reg(UInt())
    counter := counter + 1.U
    ```
        * will increment over time, but may need to worry about reset & bitwidth

## More Reason to Avoid `var`
* Reassignments can do weird things to your Chisel design that make it hard to debug 
* Chisel tools can't outright stop use of `var`, but progress has been made in detection/warning
  * ```Source has escaped the scope of the when in which it was constructed.```
* We have now covered `map`,`reduce`, and others, so not much need

In [ ]:
class DangerousVar extends Module {
    val io = IO(new Bundle {
        val in = Input(SInt(8.W))
        val out = Output(SInt(8.W))
    })
    var w = WireInit(io.in)
    when (io.in < 0.S) {
        w := 0.S   // what if typo: w = 0.S
    }
    io.out := w
}

println(getVerilog(new DangerousVar))

## Use Mutable Collections Sparingly

* We used them for HW3 because we hadn't covered functional programming yet
* Now with functional programming, can often avoid the need
  * If things are independent, can use `map`, `foreach` (for Chisel connections), or `.tabulate` (to populate a `Seq`)
  * If some sort of loop-carried dependence, can use `foldLeft`
* Example of incrementing all values
    * **Gross**
        ```scala
        val a = ArrayBuffer.tabulate(5)(_.toInt)
        for (i <- 0 until 5)
          a(i) += 1
        ```
        * Uses mutation and iteration is a distraction
    * **Better**
        ```scala
        val orig = Seq.tabulate(5)(_.toInt)
        val incremented = orig.map{ _+1 }
        ```
        * No mutation or unnecessary iteration



## Reduce Number of Special Cases

* Saw some submissions which hardcoded cases for every parameter value anticipated
  * Not scalable if number of parameter values is large
* In general, look to reduce special cases in code
  * Many constructs (e.g. `foldLeft`) work gracefully with 0 elements, so just need to handle _sufficiently general_ case
  * If you do need to handle a special case, try to limit it to only one
    * If you need more, see if you can't _generalize_

## How Does `for` or `foreach` Interact with Chisel?

* Both `for` and `foreach` impact Scala execution when constructing the Chisel design
* Typically they are best used for creating arbitrary number of connections
  * The benefit is the side effect of connection(s) being made
  * If a result is the goal, then `map` is probably a better fit
* Style wise typically prefer...
  * `foreach` if collection/range already exists
  * `for` if creating range or need index variable
    * Yes, could use `.zipWithIndex` with `foreach`, but cumbersome
  

## Avoid Unnecessary Extra Logic

* Although CAD tools can often optimize away innefficient logic, still make modest effort
  * Simpler logic can also be simpler to read/maintain
* For working with 2D grids, we often saw `%` and `/` to pull out row & column indices from a single counter
  * Consider using 2 counters (1 for row & 1 for column)
  * `%` and `/` are quite expensive in hardware, so avoid if possible
  * Also saw `*` for generating single index, but may also be avoidable
* For accessing bits or moving bits, saw `<<` and `&` (with masks)
  * Necessary in software, but not in Chisel
  * Chisel has bit select `x(hi,lo)`, `tail` and `head` to select
  * Chisel has `Cat` to put them together

## Chisel Style - Avoid Declaring Significant HW inside `when`

* Hardware declared inside a `when` block is always instantiated/exists
  * The conditional aspects of `when` only control when connections to it are active (via muxes)
  * By contrast, a declaration inside a Scala `if` may not be instantiated
* Arguably, sometimes more clear to instantiate things outside to clarify intent

In [ ]:
class CounterWhenDemo extends Module {
    val io = IO(new Bundle {
        val en = Input(Bool())
        val in = Input(UInt(8.W))
        val out = Output(UInt(8.W))
    })
    io.out := 0.U
    when (io.en) {
        val (count, wrap) = Counter(0 until 4)
        io.out := count
    }
}

println(getVerilog(new CounterWhenDemo))

## Scala Style - Use either `until` or `to`

* Scala provides both `until` (exclusive bounds) and `to` (inclusive bounds)
  * Thus, usually shouldn't need to have `n-1` or `n+1` in a bound
* Can convert `0 to n-1` to `0 until n`

In [ ]:
val n = 4
(0 until n) foreach println

## When to use `require` vs `assert`?

* Part of confusion is which `assert` (Chisel or Scala) gets called?
* Chisel `assert` checks value in _simulation,_ but not during _construction_
  * Emits non-synthesizable Verilog
  * Chisel one is used when result is `Bool` (i.e. result of Chisel comparison)
  * Can also customize assertion with failure message
* Scala `assert` checks during _construction_ but not _simulation_
  * Will be used if result is `Boolean` (i.e. result of Scala comparison)
* Recommend using `require` instead of Scala `assert`
  * Both `require` and `assert` built into Scala and evaluated at run time
  * Stylistically, `require` is for checking input sanity while `assert` is for checking internal consistency
    * Can also use flags to remove Scala `assert` (but not `require`) at compile time to reduce binary size

## Playing with Different Types of `assert`

In [ ]:
class CheckNonZero(width: Int) extends Module {
    val io = IO(new Bundle {
        val in = Input(UInt(width.W))
        val out = Output(UInt(width.W))
    })
    require(width > 0)
    assert(io.in > 0.U, "saw 0 input")
    io.out := io.in
}

println(getVerilog(new CheckNonZero(8)))

## "Flattening" & "Unflattening" Bundles

* Chisel generates a separate memory for each field of a `Bundle` (or element of a `Vec`)
  * This is usually what you want, so good default
  * However, sometimes, you may want to keep them all as one memory
* Can use `.getWidth` on a Bundle instance to see how many bits it is
* Can use `.asTypeOf` to cast bits into desired Bundle (sometimes called _reverse concatenation_)

In [ ]:
class Pair extends Bundle {
    val a = UInt(1.W)
    val b = UInt(7.W)
}

class MemCohesion extends Module {
    val io = IO(new Bundle {
        val addr = Input(UInt(8.W))
        val out = Output(new Pair())
    })
    val m = Mem(256, new Pair)
    io.out := m(io.addr)
//     val m = Mem(256, UInt((new Pair).getWidth.W))
//     io.out := m(io.addr).asTypeOf(new Pair)
}

println(getVerilog(new MemCohesion))